# Petit Photoshop

Importer les librairies suivantes :

In [1]:
import tkinter as tk
import PIL as pil
from PIL import Image
from PIL import ImageTk 
from tkinter import filedialog
from tkinter import simpledialog

Ajouter une fonction `nbrCol(matrice)` qui retourne le nombre de colonnes d'une matrice 
et une fonction `nbrLig(matrice)` qui retourne le nombre de lignes d'une matrice

In [2]:
def nbrCol(matrice):
    return(len(matrice[0]))

def nbrLig(matrice):
    return len(matrice)

Ajouter la fonction suivante à votre code, on ne demande pas de la comprendre en détail.
Elle permet de stocker la matrice de pixels passée en paramètre `matPix` dans un fichier spécifié par `filename`.

In [3]:
def saving(matPix, filename):
    toSave=pil.Image.new("RGBA",(nbrCol(matPix),nbrLig(matPix)))
    for i in range(nbrCol(matPix)):
        for j in range(nbrLig(matPix)):
            toSave.putpixel((i,j),matPix[j][i])
    toSave.save(filename)

Ajouter la fonction `loading` à votre code. Là non plus on ne demande pas de comprendre la fonction en détail.
Elle prend une image en entrée et retourne une matrice de pixels.

In [4]:
def loading(filename):
    toLoad=pil.Image.open(filename)
    mat=[[(255,255,255,255)]*toLoad.size[0] for k in range(toLoad.size[1])]
    for i in range(toLoad.size[1]):
        for j in range(toLoad.size[0]):
            mat[i][j]=toLoad.getpixel((j,i))
    return mat

Ajouter la fonction `charger` qui permet d'afficher une image dans un canevas dans la colonne 1 d'une grid.

In [5]:
create=True
nomImgCourante=""
nomImgDebut = ""

def charger(widg):
    global create
    global photo
    global img
    global canvas
    global dessin
    global nomImgCourante
    global nomImgDebut
    filename= filedialog.askopenfile(mode='rb', title='Choose a file')
    img = pil.Image.open(filename)
    nomImgCourante=filename.name
    nomImgDebut = filename.name
    photo = ImageTk.PhotoImage(img)
    if create:    
        canvas = tk.Canvas(widg, width = img.size[0], height = img.size[1])
        dessin = canvas.create_image(0,0,anchor = tk.NW, image=photo)
        canvas.grid(row=0,column=1,rowspan=4,columnspan=2)
        create=False
        
    else:
        canvas.grid_forget()
        canvas = tk.Canvas(widg, width = img.size[0], height = img.size[1])
        dessin=canvas.create_image(0,0,anchor = tk.NW, image=photo)
        canvas.grid(row=0,column=1,rowspan=4,columnspan=2)


Ajouter la fonction `modify(matrice)` qui permet de modifier une matrice de pixels au sein même d'un canevas créé par la methode `charger()`, et la fonction callback `reaffiche` qui permet de réafficher l'image de départ après lui avoir appliqué des effets.

In [6]:
def modify(matrice):
    global imgModif
    global nomImgCourante
    saving(matrice,"modif.png")
    imgModif=ImageTk.PhotoImage(file="modif.png")
    canvas.itemconfigure(dessin, image=imgModif)
    nomImgCourante="modif.png"

def reaffiche():
    global imgDebut
    global nomImgCourante
    if not create :
        imgDebut=ImageTk.PhotoImage(file=nomImgDebut)
        canvas.itemconfigure(dessin, image=imgDebut)
        nomImgCourante = nomImgDebut
    

Créez une fenêtre tkinter appelée "Mon Petit Photoshop", dans laquelle vous mettrez un bouton pour chacune des fonctions de traitement d'image que vous allez écrire. Vous mettrez égalment un bouton pour *charger* une image, un bouton pour *quitter la fenêtre* et un bouton pour *revenir à l'image de départ* en utilisant la fonction callback `reaffiche`. Vous ajouterez également un label contenant votre nom, prénom et numéro d'étudiant. Vous pourrez ajouter un bouton pour sauvegarder l'image, éventuellement sous un nouveau nom. ATTENTION : L'image est placée grâce à la méthode `grid` et elle couvre les colonnes 1 et 2 et les lignes 0,1,2 et 3.

Mise en Page 
1. Vous placerez les boutons pour les fonctions de traitement d'image dans la colonne 0 d'une grid respectivement
    aux lignes 0,1,2
1. Le bouton `Retour` (pour revenir à l'image de départ) sera placé dans la colonne 0 à l'avant dernière ligne
1. Le bouton `Charger` sera placé dans la colonne 0 dans la dernière ligne
1. Le bouton `Quitter` sera placé dans la dernière colonne et la dernière ligne
1. Le label sera placé juste sous l'image, entre les boutons `Charger` et `Quitter`
 
Si tout fonctionne, regardez la doc tkinter pour rendre ça un peu plus "joli".

In [21]:
create=True
#Fonctions auxiliaires 
    
fenetre = tk.Tk()

def fermer_fenetre():
    fenetre.destroy()

def filtre_vert():
    mat=loading(nomImgCourante)
    for i in range(nbrLig(mat)):
        for j in range(nbrCol(mat)):
            mat[i][j]=(0,mat[i][j][1],0,255)
    modify(mat)

def negatif():
    mat=loading(nomImgCourante)
    for i in range(nbrLig(mat)):
        for j in range(nbrCol(mat)):
            mat[i][j]=(255-mat[i][j][0],255-mat[i][j][1],255-mat[i][j][2],255)
    modify(mat)

def symetrique():
    mat=loading(nomImgCourante)
    matSym=[[(0,0,0,0)]*nbrCol(mat) for k in range(nbrLig(mat))]
    for i in range(nbrLig(mat)):
        for j in range(nbrCol(mat)):
            matSym[i][nbrCol(mat)-1-j]=mat[i][j]
    for i in range(nbrLig(mat)):
        for j in range(nbrCol(mat)):
            mat[i][j]=matSym[i][j]
    modify(mat)
    
def gris():
    #On utilisera la conversion CIE709 qui permet de calculer la teinte de gris qui va être affichée dans le pixel
    #La teinte affichée est : gris=0,2125*rouge + 0,0721*bleu + 0,7154*vert
    mat=loading(nomImgCourante)
    for i in range(nbrLig(mat)):
        for j in range(nbrCol(mat)):
            gris = int(0.2125*mat[i][j][0] + 0.0721*mat[i][j][1] + 0.7121*mat[i][j][2])
            mat[i][j] = (gris, gris, gris, 255)
    modify(mat)

def noirBlanc():
    mat=loading(nomImgCourante)
    for i in range(nbrLig(mat)):
        for j in range(nbrCol(mat)):
            luminosité = int((mat[i][j][0] + mat[i][j][1] + mat[i][j][2])/3)
            if luminosité > 127 :
                mat[i][j] = (255, 255, 255, 255)
            else :
                mat[i][j] = (0, 0, 0, 255)
            # un pixel est blanc quand sa luminosité est > à 127, noir sinon
    modify(mat)

def zoom():
    mat=loading(nomImgCourante)
    #créer une matrice de largeur et hauteur deux fois plus grande 
    mat_res = [[(0,0,0,0)]*nbrCol(mat)*2 for i in range (2*nbrLig(mat))]
    for i in range(nbrLig(mat)):
        for j in range(nbrCol(mat)):
            for k in range(2):
                for l in range(2):
                    mat_res[i*2+k][j*2+l] = mat[i][j]
    modify(mat_res)

#OU
#for i in range(nbrLig(mat)):
    #temp = []
    #for j in range(nbrCol(mat)):
        #temp.append(mat[i][j])
        #temp.append(mat[][j])
    #mat_res.append(temp)
    #mat_res.append(temp)

def shrink():
    mat=loading(nomImgCourante)
    nb_l = nbrLig(mat)
    nb_c = nbrCol(mat)
    #créer une matrice de largeur et hauteur deux fois plus petite 
    mat_res = [[(0,0,0,0)]*((nb_c+1)//2) for i in range ((nb_l+1)//2)]
    for i in range(nbrLig(mat_res)):
        for j in range(nbrCol(mat_res)):
            r = 0
            g = 0
            b = 0
            for k in range(2):
                if 2*i+k < nb_l:
                    for l in range(2):
                        if 2*j+l < nb_c:
                            r += mat[2*i+k][2*j+l][0]
                            g += mat[2*i+k][2*j+l][1]
                            b += mat[2*i+k][2*j+l][2]
            mat_res[i][j] = (r//4, g//4, b//4, 255)
    modify(mat_res)

def poster():
    shrink()
    zoom()

def rotate_right():
    mat=loading(nomImgCourante)
    mat_res = [[(0,0,0,0)]*nbrLig(mat) for i in range(nbrCol(mat))]
    for i in range(nbrLig(mat_res)):
        for j in range(nbrCol(mat_res)):
            mat_res[i][j] = mat[nbrLig(mat)-1-j][i]
    modify(mat_res)

def rotate_left():
    mat=loading(nomImgCourante)
    mat_res = [[(0,0,0,0)]*nbrLig(mat) for i in range(nbrCol(mat))]
    for i in range(nbrCol(mat)):
        for j in range(nbrLig(mat)):
            mat_res[i][j] = mat[j][nbrCol(mat)-1-i]
    modify(mat_res)

def modif_lum(val):
    mat = loading(nomImgCourante)
    #mat_res = [[(0,0,0,0)]*nbrLig(mat) for i in range(nbrCol(mat))]
    for i in range(nbrLig(mat)):
        for j in range(nbrCol(mat)):
            mat[i][j] = (mat[i][j][0]+val, mat[i][j][1]+val, mat[i][j][2]+val, 255)
    modify(mat)

def flou():
    mat=loading(nomImgCourante)
    nb_l = nbrLig(mat)
    nb_c = nbrCol(mat)
    for i in range(nb_l):
        for j in range(nb_c):
            r_moy = [mat[i][j][0]]
            g_moy = [mat[i][j][1]]
            b_moy = [mat[i][j][2]]
            for k in range (1,5):
                if i-k >= 0:
                    r_moy.append(mat[i-k][j][0])
                    g_moy.append(mat[i-k][j][1])
                    b_moy.append(mat[i-k][j][2])
                if i+k < nb_l:
                    r_moy.append(mat[i+k][j][0])
                    g_moy.append(mat[i+k][j][1])
                    b_moy.append(mat[i+k][j][2])
                if j-k >= 0:
                    r_moy.append(mat[i][j-k][0])
                    g_moy.append(mat[i][j-k][1])
                    b_moy.append(mat[i][j-k][2])
                if j+k < nb_c:
                    r_moy.append(mat[i][j+k][0])
                    g_moy.append(mat[i][j+k][1])
                    b_moy.append(mat[i][j+k][2])
            mat[i][j] = (sum(r_moy)//len(r_moy), sum(g_moy)//len(g_moy), sum(b_moy)//len(b_moy, 255))
    modify(mat)

#Création des Widgets
fenetre.title("Mon Petit Photoshop")
label = tk.Label(fenetre, text="LEFEVRE, Laura, 22100620")

b_filtre_vert = tk.Button(fenetre, text='Filtre vert', command= filtre_vert)

b_retour = tk.Button(fenetre, text = 'Retour', command = reaffiche)

b_charger = tk.Button(fenetre, text = 'Charger', command = lambda : charger(fenetre))

b_quitter = tk.Button(fenetre, text = 'Quitter', command = fermer_fenetre)

b_negatif = tk.Button(fenetre, text = 'Negatif', command = negatif)

b_symetric = tk.Button(fenetre, text = 'Symétrie', command = symetrique)

b_gris = tk.Button(fenetre, text = 'Niveau de gris', command = gris)

b_blackandwhite = tk.Button(fenetre, text = 'BlackAndWhite', command = noirBlanc)

b_zoom = tk.Button(fenetre, text = 'zoom', command = zoom)

b_dezoom = tk.Button(fenetre, text='dezoom', command = shrink)

b_poster = tk.Button(fenetre, text='Poster', command = poster)

b_rotate_right = tk.Button(fenetre, text='Rotate right', command = rotate_right)

b_rotate_left = tk.Button(fenetre, text='Rotate left', command = rotate_left)

slider = tk.Scale(fenetre, from_=-255, to_=255)
b_slider = tk.Button(fenetre, text='luminosite', command = lambda: modif_lum(slider.get()))

b_flou = tk.Button(fenetre, text='Flou', command=flou)

#Positionnement des Widgets
label.grid(column = 2, row = 7)

b_filtre_vert.grid(column = 0, row = 0)

b_negatif.grid(column = 0, row = 1)

b_symetric.grid(column = 0, row = 2)

b_retour.grid(column = 0, row = 6)

b_charger.grid(column = 1, row = 7)

b_quitter.grid(column = 3, row = 7)

b_gris.grid(column=0, row = 3)

b_blackandwhite.grid(column = 0, row=4)

b_zoom.grid(column = 0, row = 5)

b_dezoom.grid(column = 0, row = 7)

b_poster.grid(column = 4, row = 1)

b_rotate_right.grid(column = 4, row = 2)

b_rotate_left.grid(column = 4, row = 3)

slider.grid(column = 4, row = 4)

b_slider.grid(column = 4, row = 5)

b_flou.grid(column = 4, row = 0)

#Lancement de la boucle 
fenetre.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Eleve\miniconda3\envs\l1-python\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Eleve\AppData\Local\Temp/ipykernel_8728/2916206748.py", line 151, in flou
    mat[i][j] = (sum(r_moy)//len(r_moy), sum(g_moy)//len(g_moy), sum(b_moy)//len(b_moy, 255))
TypeError: len() takes exactly one argument (2 given)
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Eleve\miniconda3\envs\l1-python\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Eleve\AppData\Local\Temp/ipykernel_8728/2916206748.py", line 151, in flou
    mat[i][j] = (sum(r_moy)//len(r_moy), sum(g_moy)//len(g_moy), sum(b_moy)//len(b_moy, 255))
TypeError: len() takes exactly one argument (2 given)


Modifiez les fonctions du TD précédent pour qu'elles deviennent des fonctions callback qui s'appliquent directement sur l'image dans le canevas.

In [7]:
def filtre_vert():
    mat=loading(nomImgCourante)
    for i in range(nbrLig(mat)):
        for j in range(nbrCol(mat)):
            mat[i][j]=(0,mat[i][j][1],0,255)
    modify(mat)
            
def negatif():
    mat=loading(nomImgCourante)
    for i in range(nbrLig(mat)):
        for j in range(nbrCol(mat)):
            mat[i][j]=(255-mat[i][j][0],255-mat[i][j][1],255-mat[i][j][2],255)
    modify(mat)
            
def symetrique():
    mat=loading(nomImgCourante)
    matSym=[[(0,0,0,0)]*nbrCol(mat) for k in range(nbrLig(mat))]
    for i in range(nbrLig(mat)):
        for j in range(nbrCol(mat)):
            matSym[i][nbrCol(mat)-1-j]=mat[i][j]
    for i in range(nbrLig(mat)):
        for j in range(nbrCol(mat)):
            mat[i][j]=matSym[i][j]
    modify(mat)

Faire une fonction qui permet de transformer une image en couleur en une image en niveau de gris.

In [8]:
#Image en couleur => image en niveau de gris

#Pour un pixel :
#   - chaque composante R,G,B a la même valeur
#   - cette valeur correspond à une proportion des valeurs des composantes R,G,B
# Par exemple : norme CIE709 -> 0,2125xR + 0,072xB + 0,7154xG

# luminosité d'un pixel = moyenne des trois composantes R,G,B

In [9]:
def gris():
    #On utilisera la conversion CIE709 qui permet de calculer la teinte de gris qui va être affichée dans le pixel
    #La teinte affichée est : gris=0,2125*rouge + 0,0721*bleu + 0,7154*vert
    mat=loading(nomImgCourante)
    for i in range(nbrLig(mat)):
        for j in range(nbrCol(mat)):
        pass
            # calcul de la teinte de gris du pixel (CIE709)
    modify(mat)


IndentationError: expected an indented block (Temp/ipykernel_8728/1166345872.py, line 7)

Faire une fonction qui permet de transformer une image en couleur en une image en noir et blanc.

In [ ]:
def noirBlanc():
    mat=loading(nomImgCourante)
    for i in range(nbrLig(mat)):
        for j in range(nbrCol(mat)):
            luminosité = int((mat[i][j][0] + mat[i][j][1] + mat[i][j][2])/3)
            if luminosité > 127 :
                mat[i][j] = (255, 255, 255, 255)
            else :
                mat[i][j] = (0, 0, 0, 255)
            # un pixel est blanc quand sa luminosité est > à 127, noir sinon
    modify(mat)

Donner une fonction qui multiplie par 4 la définition d'une image (chaque pixel donne un carré de 4 pixels).

In [ ]:
def zoom():
    mat=loading(nomImgCourante)
    #créer une matrice de largeur et hauteur deux fois plus grande 
    for i in range(nbrLig(mat)):
        for j in range(nbrCol(mat)):
            mat.insert(mat[i+1][j], mat[i][j])
            mat.insert(mat[i][j+1], mat[i][j])
            mat.insert(mat[i+1][j+1], mat[i][j])
    modify(mat)

Donner une fonction qui divise par 4 la définition d'une image. Chaque carré de 4 pixels est remplacé par 1 pixel dont la valeur est la moyenne de ces 4 pixels.

In [ ]:
def shrink():
    mat=loading(nomImgCourante)
    #créer une matrice de largeur et hauteur deux fois plus petite 
    for i in range(nbrLig(mat)):
        for j in range(nbrCol(mat)):
            pass
    modify(mat)

Combiner les fonctions `zoom` et `shrink` pour obtenir un effet poster (effet de perte de résolution).

In [ ]:
def poster():
    mat=loading(nomImgCourante)
    modify(mat)

Donner une fonction qui effectue une rotation d'une image vers la droite.

In [ ]:
def rotate():
    mat=loading(nomImgCourante)
    modify(mat)

Voici quelques fonctions que vous pouvez implémenter si vous en avez le temps:
1. modification de la luminosité d'une image (avec un slider pour la sélection)
1. flou de l'image, chaque pixel est remplacé par la moyenne de ses voisins et de lui-même
1. bruitage de l'image, chaque pixel est modifié par une valeur aléatoire
1. implémenter une méthode de dithering pour obtenir une image en noir et blanc plus subtile (au lieu de prendre un seuil
à 127, on tire aléaoirement la valeur du pixel avec une probabilité qui dépend de sa luminosité)
1. mode stylo, qui permet avec la souris d'écrire des pixels noirs sur l'image 